# Classification using augmentation of images

In [2]:
import cv2
import tensorflow as tf
import os
import numpy as np

### Load images

In [3]:
def load_img(indir):
    samples = []
    labels = []
    i = 0
    for class_dir in os.listdir(indir):
        the_class = class_dir
        for file in os.listdir(indir+'/'+class_dir):
            image = cv2.imread("{}/{}/{}".format(indir,class_dir,file))
            image = cv2.resize(image, (128,128))
            samples.append(image)
            labels.append(the_class)
            if i%10==0: print('.',end='')
            i = i + 1
    samples = np.array(samples)
    labels = np.array(labels)
    return samples,labels
samples, labels = load_img('dataset')
print('loaded',len(samples),' samples')
print('classes',set(labels))

.....................loaded 202  samples
classes {'cats', 'dogs'}


### Build model

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

model = Sequential()
model.add(Conv2D(16, (3, 3), padding="same",input_shape=(128,128,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("sigmoid"))
model.add(Dense(2))
model.add(Activation("softmax"))

#model.summary()

### Encode labels

In [5]:
from sklearn.preprocessing import LabelEncoder 
encoder = LabelEncoder() # encoder
intlabels = encoder.fit_transform(labels)
labels = tf.keras.utils.to_categorical(intlabels)
print("One-hot labels",labels.shape)

One-hot labels (202, 2)


## Create genarator

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
augmentator = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

## Use the generator in training

In [8]:
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
print(samples.shape)
import sklearn.model_selection
(trainSamples, testSamples, trainLabels, testLabels) = sklearn.model_selection.train_test_split(samples,labels,random_state=42)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

checkpt = ModelCheckpoint (filepath='augm_model.h5', save_best_only=True)
estopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

EPOCHS = 100
#H = model.fit(trainSamples, trainLabels, epochs=EPOCHS, batch_size=128, validation_data=(testSamples,testLabels),
#             callbacks = [estopping,checkpt]) 
print(trainSamples.shape)
print(testSamples.shape)

BATCH_SIZE = 20
H = model.fit(augmentator.flow(trainSamples, trainLabels, batch_size=BATCH_SIZE),
            epochs=EPOCHS, validation_data=(testSamples,testLabels),
            #callbacks = [estopping,checkpt],
              steps_per_epoch=len(trainSamples)//BATCH_SIZE
             ) 

print("Done")

(202, 128, 128, 3)
(151, 128, 128, 3)
(51, 128, 128, 3)
Epoch 1/100
7/7 [==============================] - 3s 437ms/step - loss: 1.0567 - accuracy: 0.5344 - val_loss: 0.7225 - val_accuracy: 0.5294
Epoch 2/100
7/7 [==============================] - 2s 330ms/step - loss: 0.7531 - accuracy: 0.5115 - val_loss: 0.7041 - val_accuracy: 0.5294
Epoch 3/100
7/7 [==============================] - 2s 337ms/step - loss: 0.7122 - accuracy: 0.4809 - val_loss: 0.7479 - val_accuracy: 0.4706
Epoch 4/100
7/7 [==============================] - 2s 357ms/step - loss: 0.7085 - accuracy: 0.5267 - val_loss: 0.7015 - val_accuracy: 0.4706
Epoch 5/100
7/7 [==============================] - 3s 387ms/step - loss: 0.6988 - accuracy: 0.4714 - val_loss: 0.6926 - val_accuracy: 0.5294
Epoch 6/100
7/7 [==============================] - 2s 315ms/step - loss: 0.6951 - accuracy: 0.5038 - val_loss: 0.6945 - val_accuracy: 0.4706
Epoch 7/100
7/7 [==============================] - 2s 343ms/step - loss: 0.6947 - accuracy: 0.4962

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, cohen_kappa_score
testResults = model.predict(testSamples)

print(confusion_matrix(testLabels.argmax(axis=1), testResults.argmax(axis=1)))
print(classification_report(testLabels.argmax(axis=1), testResults.argmax(axis=1)))
print("CNN Accuracy: {:.2f}".format(accuracy_score(testLabels.argmax(axis=1), testResults.argmax(axis=1))))
print("Cohen's Kappa: {:.2f}".format(cohen_kappa_score(testLabels.argmax(axis=1), testResults.argmax(axis=1))))
